In [10]:
# Sample dataset: https://www.kaggle.com/chandramoulinaidu/spam-classification-for-basic-nlp

import pandas as pd

#dataframe init
df = pd.read_csv('Spam Email raw text for NLP.csv')

df.head()

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6


In [11]:
df.tail()

,CATEGORY,MESSAGE,FILE_NAME
5791,0,"I'm one of the 30,000 but it's not working ver...",00609.dd49926ce94a1ea328cce9b62825bc97
5792,0,Damien Morton quoted:\n\n>W3C approves HTML 4 ...,00957.e0b56b117f3ec5f85e432a9d2a47801f
5793,0,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n\n\...",01127.841233b48eceb74a825417d8d918abf8
5794,0,"Once upon a time, Manfred wrote :\n\n\n\n> I w...",01178.5c977dff972cd6eef64d4173b90307f0
5795,0,"If you run Pick, and then use the ""New FTOC"" b...",00747.352d424267d36975a7b40b85ffd0885e


In [14]:
df['CATEGORY'].value_counts()

0    3900
1    1896
Name: CATEGORY, dtype: int64

In [15]:
import nltk
#natural language tool kit

nltk.download('wordnet')
nltk.download('stopwords')



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
test_message = "Hey,, GGggGG feet it going? <HTML><bads> bads 'randoms' badly"
#initial test
test_message_tokenized = tokenizer.tokenize(test_message)
test_message_tokenized

['Hey',
 'GGggGG',
 'feet',
 'it',
 'going',
 'HTML',
 'bads',
 'bads',
 'randoms',
 'badly']

In [17]:
test_message_lowercased = [t.lower() for t in test_message_tokenized]
test_message_lowercased

['hey',
 'gggggg',
 'feet',
 'it',
 'going',
 'html',
 'bads',
 'bads',
 'randoms',
 'badly']

In [18]:
#lemmatizing
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
test_message_lemmatized_tokens = [lemmatizer.lemmatize(t) for t in test_message_lowercased]
test_message_lemmatized_tokens


['hey',
 'gggggg',
 'foot',
 'it',
 'going',
 'html',
 'bad',
 'bad',
 'randoms',
 'badly']

In [19]:
#now remove stop words
from nltk.corpus import stopwords

stopwords = stopwords.words('english')
test_message_useful_tokens = [t for t in test_message_lemmatized_tokens if t not in stopwords]
test_message_useful_tokens
#if not in stop words, then it is a useful word


['hey', 'gggggg', 'foot', 'going', 'html', 'bad', 'bad', 'randoms', 'badly']

In [20]:
def message_to_token_list(s):
  tokens = tokenizer.tokenize(s)
  lowercased_tokens = [t.lower() for t in tokens]
  lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lowercased_tokens]
  useful_tokens = [t for t in lemmatized_tokens if t not in stopwords]

  return useful_tokens
  #function for all of text preprocessing
message_to_token_list(test_message)

['hey', 'gggggg', 'foot', 'going', 'html', 'bad', 'bad', 'randoms', 'badly']

In [21]:
df = df.sample(frac=1, random_state=1)
df = df.reset_index(drop=True)

split_index = int(len(df) * 0.8)
train_df, test_df = df[:split_index], df[split_index:]

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df, test_df

(      CATEGORY                                            MESSAGE  \
 0            1  \n\n<HTML><FONT  BACK="#ffffff" style="BACKGRO...   
 1            1  <html><body bgColor="#CCCCCC" topmargin=1 onMo...   
 2            0  Quoting Paul Linehan (plinehan@yahoo.com):\n\n...   
 3            0  <a href=http://www.aaronsw.com/weblog/>\n\nAar...   
 4            0  Oh yeah, the link for more info:\n\n\n\nhttp:/...   
 ...        ...                                                ...   
 4631         0  Gregory Alan Bolcer:\n\n>I'm not sure since I ...   
 4632         1  New Account For: zzzz@spamassassin.taint.org\n...   
 4633         0  >>>>> "O" == Owen Byrne <owen@permafrost.net> ...   
 4634         0  This is an automated response to a message you...   
 4635         0  http://www.ouchytheclown.com/welcome.html\n\n\...   
 
                                    FILE_NAME  
 0     00118.141d803810acd9d4fc23db103dddfcd9  
 1     00463.0bc4e08af0529dd773d9f10f922547db  
 2     00358.8

In [22]:
token_counter = {}

for message in train_df['MESSAGE']:
  message_as_token_list = message_to_token_list(message)

  for token in message_as_token_list:
    if token in token_counter:
      token_counter[token] += 1
    else:
      token_counter[token] = 1

len(token_counter)

86415

In [23]:
#maintain valid tokens
def keep_token(processed_token, threshold):
  if processed_token not in token_counter:
    return False
  else:
    return token_counter[processed_token] > threshold

#test
keep_token('random', 100)

False

In [24]:
features = set()

#bag of words
for token in token_counter:
  if keep_token(token, 10000):
    features.add(token)

features

{'3d', 'b', 'br', 'com', 'font', 'http', 'p', 'size', 'td', 'tr'}

In [25]:
#token-index mapping
token_to_index_mapping = {t:i for t, i, in zip(features, range(len(features)))}
token_to_index_mapping

{'size': 0,
 'p': 1,
 'font': 2,
 'com': 3,
 'br': 4,
 'b': 5,
 'tr': 6,
 '3d': 7,
 'http': 8,
 'td': 9}

In [26]:
#temp
message_to_token_list('3d b <br> .com bad font font com randoms')

['3d', 'b', 'br', 'com', 'bad', 'font', 'font', 'com', 'randoms']

In [27]:
import numpy as np

#vector representation of messages
def message_to_count_vector(message):
  count_vector = np.zeros(len(features))

  processed_list_of_tokens = message_to_token_list(message)

  for token in processed_list_of_tokens:
    if token not in features:
      continue
    index = token_to_index_mapping[token]
    count_vector[index] += 1

  return count_vector

message_to_count_vector('3d b <br> .com bad font font com randoms')

array([0., 0., 2., 2., 1., 1., 0., 1., 0., 0.])

In [28]:
message_to_count_vector(train_df['MESSAGE'].iloc[0])
#read first message test

array([ 2.,  0.,  4.,  9., 33.,  2.,  0.,  0.,  6.,  0.])

In [29]:
train_df.iloc[3]

CATEGORY                                                     0
MESSAGE      <a href=http://www.aaronsw.com/weblog/>\n\nAar...
FILE_NAME               01274.0d083a2d3b30061efdc2cc73ee9e76e3
Name: 3, dtype: object

In [30]:
def df_to_x_y(dff):
  y = dff['CATEGORY'].to_numpy().astype(int)

  message_col = dff['MESSAGE']
  count_vectors = []

  for message in message_col:
    count_vector = message_to_count_vector(message)
    count_vectors.append(count_vector)

  x = np.array(count_vectors).astype(int)

  return x, y

In [31]:
x_train, y_train = df_to_x_y(train_df)
x_test, y_test = df_to_x_y(test_df)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4636, 10), (4636,), (1160, 10), (1160,))

In [32]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(x_train)

x_train, x_test = scaler.transform(x_train), scaler.transform(x_test)

x_train

array([[0.00732601, 0.        , 0.00245851, ..., 0.        , 0.025     ,
        0.        ],
       [0.02564103, 0.04065041, 0.00860479, ..., 0.        , 0.00833333,
        0.01346801],
       [0.        , 0.00406504, 0.        , ..., 0.        , 0.00416667,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.00416667,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00416667,
        0.        ],
       [0.        , 0.00406504, 0.        , ..., 0.        , 0.0125    ,
        0.        ]])

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression().fit(x_train, y_train)
print(classification_report(y_test, lr.predict(x_test)))

              precision    recall  f1-score   support

           0       0.76      1.00      0.86       788
           1       0.99      0.32      0.48       372

    accuracy                           0.78      1160
   macro avg       0.87      0.66      0.67      1160
weighted avg       0.83      0.78      0.74      1160



In [35]:
# comparing logistic regression to random forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier().fit(x_train, y_train)
print(classification_report(y_test, rf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.83      0.98      0.89       788
           1       0.92      0.56      0.70       372

    accuracy                           0.84      1160
   macro avg       0.87      0.77      0.80      1160
weighted avg       0.86      0.84      0.83      1160

